# MovieLens Data Cleaning
***

## Table of Contents
***

1. [Introduction](#intro)
1. [Loading](#loading)
1. [Ratings](#ratings)
1. [Movies](#movies)
1. [Links](#links)
1. [Tags](#tags)
1. [Conclusion](#conclusion)

## Introduction <a name="intro">
***
The MovieLens 100K Dataset contains four different .csv files which need to be cleaned. The data are contained in four files, `links.csv`, `movies.csv`, `ratings.csv` and `tags.csv`. Each one will be explored and cleaned trying to avoid dropping any data if possible. 

## Loading<a name="loading">
***
Let's load all the libraries and datasets.

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import requests
from bs4 import BeautifulSoup
import string
import re
from unicodedata import normalize

In [2]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
movies = pd.read_csv('ml-latest-small/movies.csv')
links = pd.read_csv('ml-latest-small/links.csv')
tags = pd.read_csv('ml-latest-small/tags.csv')

## Ratings <a name="ratings">
***
All ratings are contained in the file `ratings.csv`.

In [3]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [4]:
ratings.shape

(100004, 4)

In [5]:
ratings.isnull().sum()/len(ratings)*100

userId       0.0
movieId      0.0
rating       0.0
timestamp    0.0
dtype: float64

In [6]:
ratings.describe()

,userId,movieId,rating,timestamp
count,100004.000000,100004.000000,100004.000000,1.000040e+05
mean,347.011310,12548.664363,3.543608,1.129639e+09
std,195.163838,26369.198969,1.058064,1.916858e+08
min,1.000000,1.000000,0.500000,7.896520e+08
25%,182.000000,1028.000000,3.000000,9.658478e+08
50%,367.000000,2406.500000,4.000000,1.110422e+09
75%,520.000000,5418.000000,4.000000,1.296192e+09
max,671.000000,163949.000000,5.000000,1.476641e+09


The ratings dataset has 100,004 entries and no missing values. The lines within this file are ordered first by userId, then, within user, by movieId. Ratings are made on a 5-star scale, with half-star increments (0.5 stars - 5.0 stars). Now let's use the timestamp to create a readable date column and get more information about each rating event.

In [7]:
def convert_datetime(df):
    # Use timestamp to add datetime column
    df['date'] = pd.to_datetime(df['timestamp'], unit='s')

    # Create Float columns to split datetime objects
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['hour'] = df['date'].dt.hour

    # Drop timestamp column
    df.drop('timestamp',axis=1,inplace=True)

def save_dataset(df,name):
    # save df to cleaned_datasets folder
    df.to_csv('ml-latest-small/'+name, encoding='utf-8', index=False)

In [8]:
convert_datetime(ratings)
save_dataset(ratings,'cleaned_ratings.csv')

Let's leave the date column for the data analysis step.

## Movies <a name="movies">
***
Movie information is contained in the file `movies.csv`. 

In [9]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [10]:
movies.shape

(9125, 3)

In [11]:
movies.isnull().sum()/len(movies)*100

movieId    0.0
title      0.0
genres     0.0
dtype: float64

At first glance the data seems not to be missing information. The dataset includes 9,125 entries displaying the movieId, title and genre. The movie title includes the year of release in parentheses. A movie can have multiple genres. There is some data cleaning work to do on this dataset. First, let's create a column for the years of the movies.

In [12]:
movies['year'] = movies['title'].str.rstrip().str[-5:-1]

In [13]:
missing_years = movies.iloc[movies[movies['year'].str.isdigit() == False].index,:].reset_index()
missing_years

,index,movieId,title,genres,year
0,8505,108548,"Big Bang Theory, The (2007-)",Comedy,007-
1,9017,143410,Hyena Road,(no genres listed),Roa
2,9063,151307,The Lovers and the Despot,(no genres listed),espo
3,9118,162376,Stranger Things,Drama,hing
4,9124,164979,"Women of '69, Unboxed",Documentary,boxe


The above movies do not have the year they were published. Since there are less than 5 we can easily do this manually so that there is no need of dropping valuable data. However, it is easier just to scrape the data using BeautifulSoup package since there might be the need to do it again. As well, we can see that some movies do not have a genre listed so we will explore genres afterwards.

In [14]:
def make_request(imdbId):
    # imdbId should be 7 characters. If less pad with zeroes.
    imdbId = str(imdbId).zfill(7) 
    # data url to make request, used imdb website
    data_url = 'https://www.imdb.com/title/tt' + imdbId 
    # sends a GET request to specific url
    r  = requests.get(data_url)
    # return response object
    return r

def scrape_data(r, name, attrs):
    # extract text from response
    data = r.text
    # HTML string to be parsed
    soup = BeautifulSoup(data, 'html.parser')
    # find the element by tag
    name_box = soup.find(name, attrs=attrs)
    return name_box

def clean_year(name_box):
    # remove leading and trailing whitespace and tokenize on white space
    info = name_box.text.strip().split()
    # check if each string is 4 digits 
    year = [s for s in info if len(s)==4 if s.isdigit()]
    if not year:
        # slice year from last string in list
        year = [info[-1][1:5]]
        # check if lenght of year is less than 4
        if len(year[0]) < 4:
            # slice year from string
            year = [info[-2][1:5]]
    return year[0]

In [15]:
# index and imdbId for movies with missing years
years_ids = missing_years.merge(links,on='movieId',how='inner')[['index','imdbId']]

In [16]:
movie_ids = years_ids
# where we can find the year needed 
name = 'div'
attributes = {'class': 'subtext'}
years = []
# iterate over ids using a for loop
for each in movie_ids.values:
    # make a request to specific imdbId
    response = make_request(each[1])
    # scrape specific data
    name_box = scrape_data(response,name,attributes)
    # clean the year data
    year = clean_year(name_box)
    # append to list
    years.append(year)

# dictionary with data for df
d = {'index': years_ids['index'], 'year': years}
# create dataframe 
years_data = pd.DataFrame(d)

In [17]:
# replace missing years with scraped years 
movies.iloc[years_data['index'].values,3] = years_data['year'].values 
# check for missing years and print
print('There are {} missing years'.format(len(movies.iloc[movies[movies['year'].str.isdigit() == False].index,:])))

There are 0 missing years


In [18]:
# change the year column type to int
movies['year'] = movies['year'].astype('int')

In [19]:
movies.head()

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995


As previously shown, some movies have multiple genres and missing values. So first let's check which movies that have *(no genres listed)* and double check the website to see if there is no genre information.

In [20]:
# check which movies have genre (no genres listed)
missing_genre = movies[movies['genres']=='(no genres listed)'].reset_index()
missing_genre.head(10)

,index,movieId,title,genres,year
0,7724,83829,Scorpio Rising (1964),(no genres listed),1964
1,8732,117192,Doctor Who: The Time of the Doctor (2013),(no genres listed),2013
2,8784,122888,Ben-hur (2016),(no genres listed),2016
3,8799,126106,Beastie Boys: Sabotage (1994),(no genres listed),1994
4,8829,128616,As We Were Dreaming (2015),(no genres listed),2015
5,8830,128620,Victoria (2015),(no genres listed),2015
6,8834,129250,Superfast! (2015),(no genres listed),2015
7,8882,132549,Grandma (2015),(no genres listed),2015
8,8887,132952,Sarfarosh (1999),(no genres listed),1999
9,8901,134025,Open Secret (2013),(no genres listed),2013


There are 18 movies with no genre. Just by doing a simple search shows that the first movie on the list [Scorpio Rising](https://www.imdb.com/title/tt0058555/) is a *Short, Music*. So let's try to scrape all the missing genres to update our dataset.

In [21]:
genre_ids = missing_genre.merge(links,on='movieId',how='inner')[['index','imdbId']]

In [22]:
movie_ids = genre_ids
name = 'span'
attributes = {'class': 'itemprop'}
genres = []
# iterate over ids using a for loop
for each in movie_ids.values:
    # make a request to specific imdbId
    response = make_request(each[1])
    # scrape specific data
    name_box = scrape_data(response,name,attributes)
    # append all genres to list
    genres.append(name_box.text.rstrip())

# dictionary with data for df
d = {'index': genre_ids['index'], 'genre': genres}
# create dataframe
genre_data = pd.DataFrame(d)

In [23]:
genre_data.groupby('genre').count()

,index
genre,
Action,3
Adventure,1
Comedy,4
Crime,2
Documentary,3
Drama,1
Short,4


In [24]:
# replace missing genres with scraped genres
movies.iloc[genre_data['index'],2] = genre_data['genre'].values

Now let's binary encode the genre column by making each genre a column.

In [25]:
# split each genre and count them in a dictionary
genre_dict = movies.genres.str.split('|').apply(Counter)
# convert each movie's genres to dataframe and fill with zeroes if not present
genre = pd.DataFrame.from_records(genre_dict).fillna(value=0)

In [26]:
# concatenate the movie and genre df
cleaned_movies = pd.concat([movies,genre],axis=1)
# drop genres string column 
cleaned_movies.drop('genres',axis=1,inplace=True)
cleaned_movies.head()

,movieId,title,year,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Short,Thriller,War,Western
0,1,Toy Story (1995),1995,0.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),1995,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),1995,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),1995,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II (1995),1995,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
save_dataset(cleaned_movies,'cleaned_movies.csv')

Let's leave the names of each of the movies for now since it will help for the data analysis step.

## Links <a name="links">
***
Identifiers that can be used to link to other sources of movie data are contained in the file `links.csv`.

In [28]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [29]:
links.shape

(9125, 3)

In [30]:
links.describe()

,movieId,imdbId,tmdbId
count,9125.000000,9.125000e+03,9112.000000
mean,31123.291836,4.798244e+05,39104.545544
std,40782.633604,7.431774e+05,62814.519801
min,1.000000,4.170000e+02,2.000000
25%,2850.000000,8.884600e+04,9451.750000
50%,6290.000000,1.197780e+05,15852.000000
75%,56274.000000,4.284410e+05,39160.500000
max,164979.000000,5.794766e+06,416437.000000


In [31]:
links.isnull().sum()/len(links)*100

movieId    0.000000
imdbId     0.000000
tmdbId     0.142466
dtype: float64

In [32]:
save_dataset(links,'cleaned_links.csv')

There are a few missing values in *tmdbId* column. However, the *imdbId* column is complete and we can use IMBD's information for now. 

## Tags <a name="tags">
***
All tags are contained in the file `tags.csv`.

In [33]:
tags.head()

,userId,movieId,tag,timestamp
0,15,339,sandra 'boring' bullock,1138537770
1,15,1955,dentist,1193435061
2,15,7478,Cambodia,1170560997
3,15,32892,Russian,1170626366
4,15,34162,forgettable,1141391765


In [34]:
tags.shape

(1296, 4)

In [35]:
tags.describe()

,userId,movieId,timestamp
count,1296.000000,1296.000000,1.296000e+03
mean,417.026235,42278.949846,1.324337e+09
std,142.183440,44628.345568,1.093886e+08
min,15.000000,1.000000,1.137217e+09
25%,346.000000,2988.000000,1.243455e+09
50%,431.000000,26958.500000,1.342849e+09
75%,547.000000,72268.250000,1.440380e+09
max,663.000000,164979.000000,1.476651e+09


In [36]:
tags.isnull().sum()/len(tags)*100

userId       0.0
movieId      0.0
tag          0.0
timestamp    0.0
dtype: float64

Tags are user-generated metadata about movies. Each tag is typically a single word or short phrase. The meaning, value, and purpose of a particular tag is determined by each user. There are 1,296 entries on the tags dataset and no missing data. Let's clean the strings in the tag column.

In [37]:
def clean_lines(lines):
    cleaned = list()
    # prepare regex for char filtering
    re_print = re.compile('[^%s]' % re.escape(string.printable))
    # prepare table for removing punctuation
    table = str.maketrans('', '', string.punctuation)
    for line in lines:
        # normalize unicode characters
        line = normalize('NFD', line).encode('ascii', 'ignore')
        line = line.decode('UTF-8')
        # tokenize on white space
        line = line.split()
        # remove punctuation from each token
        line = [word.translate(table) for word in line]
        #store as string
        cleaned.append(' '.join(line))
    return cleaned

In [38]:
cleaned = clean_lines(tags['tag'].str.lower().values)
tags['tag'] = pd.Series(cleaned)
tags.head()

,userId,movieId,tag,timestamp
0,15,339,sandra boring bullock,1138537770
1,15,1955,dentist,1193435061
2,15,7478,cambodia,1170560997
3,15,32892,russian,1170626366
4,15,34162,forgettable,1141391765


In [39]:
tags[(tags['tag'].isnull()) | (tags['tag']=='')]

,userId,movieId,tag,timestamp


After cleaning each tag string, there are no missing values. Let's convert the timestamps to datetime to get more information and save the dataset.

In [40]:
convert_datetime(tags)
save_dataset(tags,'cleaned_tags.csv')

In [41]:
print('There are {} different tags.'.format(len(tags['tag'].str.lower().value_counts())))

There are 557 different tags.


Since there are many different tags, let's leave them in text for now to use them for data analysis.

## Conclusion <a name="conclusion">
***
The data was split in four different .csv files which were loaded separately. The `ratings` dataset had 100,004 entries and no missing values. Ratings were made on a 5-star scale, with half-star increments (0.5 stars - 5.0 stars). The `movies` dataset included 9,125 entries. The movie title included the year of release in parentheses which was stripped and placed on a different column. There were 18 movies with missing genres which were scrapped from each movie's website and correctly filled without the need of dropping data. As well, some of the movies had multiple genres so they were split into different columns and binary encoded.
   
The `links` dataset had identifiers that can be used to link to other sources of movie data. Basically, there were two website sources for each movie which were IMDB and TMDB. TMDB had 0.14% of missing values. However, IMDB had no missing values and it was used to scrape movie information that was missing in other datasets. So the missing values from the TMDB were no issue.

The `tags` dataset had all the tag information. Tags are user-generated metadata about movies. Each tag was typically a single word or short phrase. The meaning, value, and purpose of a particular tag was determined by each user. There were 1,296 entries on the tags dataset and no missing tags. Finally, each tag's string was cleaned by removing punctuation, normalizing characters and removing unnecessary white space. 

In conclusion, the datasets were able to maintain all the data. Whenever there was a missing value it was either scraped from the IMDB's website or filled using other movies information. New cleaned datasets were saved for further use.

### Next: [Data Analysis](https://github.com/emmpew/recommender/blob/master/data_analysis.ipynb)